In [1]:
# 01_load_buildings.ipynb

import os
import textwrap

import pandas as pd
import geopandas as gpd

gpd.options.use_pygeos = True


# Display options
pd.set_option("display.max_columns", 50)
pd.set_option("display.width", 120)


C:\Users\RyanOwens\AppData\Local\Temp\ipykernel_27520\546365827.py:9: UserWarning: pygeos support was removed in 1.0. geopandas.use_pygeos is a no-op and will be removed in geopandas 1.1.
  gpd.options.use_pygeos = True


In [2]:
# Define paths relative to repo root

REPO_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))

DATA_RAW = os.path.join(REPO_ROOT, "data", "raw")
DATA_PROCESSED = os.path.join(REPO_ROOT, "data", "processed")

PATH_BUILDING_CURRENT_ZIP = os.path.join(
    DATA_RAW, "building_current", "nyc_building_footprints_current.zip"
)

PATH_BUILDING_HISTORIC_ZIP = os.path.join(
    DATA_RAW, "building_historic", "nyc_building_footprints_historic.zip"
)

PATH_STORMWATER_GDB = os.path.join(
    DATA_RAW, "stormwater", "stormwater_moderate_current.gdb"
)


print("Repo root:", REPO_ROOT)
print("Current buildings zip:", PATH_BUILDING_CURRENT_ZIP)
print("Historic buildings zip:", PATH_BUILDING_HISTORIC_ZIP)
print("Stormwater zip:", PATH_STORMWATER_GDB)

for p in [PATH_BUILDING_CURRENT_ZIP, PATH_BUILDING_HISTORIC_ZIP, PATH_STORMWATER_GDB]:
    print(p, "exists:", os.path.exists(p))

import pyogrio
pyogrio.list_layers(PATH_STORMWATER_GDB)



Repo root: c:\Code\NYCDSA\nyc-urban-growth-planimetrics
Current buildings zip: c:\Code\NYCDSA\nyc-urban-growth-planimetrics\data\raw\building_current\nyc_building_footprints_current.zip
Historic buildings zip: c:\Code\NYCDSA\nyc-urban-growth-planimetrics\data\raw\building_historic\nyc_building_footprints_historic.zip
Stormwater zip: c:\Code\NYCDSA\nyc-urban-growth-planimetrics\data\raw\stormwater\stormwater_moderate_current.gdb
c:\Code\NYCDSA\nyc-urban-growth-planimetrics\data\raw\building_current\nyc_building_footprints_current.zip exists: True
c:\Code\NYCDSA\nyc-urban-growth-planimetrics\data\raw\building_historic\nyc_building_footprints_historic.zip exists: True
c:\Code\NYCDSA\nyc-urban-growth-planimetrics\data\raw\stormwater\stormwater_moderate_current.gdb exists: True


array([['NYC_Stormwater_Flood_Map_Moderate_Flood_2_13_inches_per_hr_with_Current_Sea_Levels',
        'MultiPolygon Z']], dtype=object)

In [3]:
def inspect_geodf(gdf: gpd.GeoDataFrame, name: str, show_head: int = 3, max_value_counts: int = 5):
    print("=" * 80)
    print(f"DATASET: {name}")
    print("=" * 80)
    print(f"Type: {type(gdf)}")
    print(f"Number of rows: {len(gdf):,}")
    print(f"Number of columns: {gdf.shape[1]}")
    print("\nCRS:", gdf.crs)
    
    # Geometry type(s)
    geom_types = gdf.geom_type.value_counts()
    print("\nGeometry types:")
    print(geom_types.head(10))
    
    print("\nColumns and dtypes:")
    print(gdf.dtypes)
    
    print("\nFirst few rows:")
    display(gdf.head(show_head))
    
    print("\nNull counts (top 15 columns by null count):")
    null_counts = gdf.isna().sum().sort_values(ascending=False)
    display(null_counts.head(15))
    
    # Try to show a few value counts for "interesting" columns
    candidate_cols = [
        col for col in gdf.columns 
        if col.lower() in ("borough", "boro_name", "nta", "nta_name", "zone", "feat_code", "type", "class")
    ]
    
    if candidate_cols:
        print("\nSample value counts for key categorical fields:")
        for col in candidate_cols:
            print(f"\nValue counts for '{col}':")
            display(gdf[col].value_counts().head(max_value_counts))
    else:
        print("\nNo obvious categorical fields found among standard names; you may want to add custom checks.")


In [4]:
def read_zipped_shapefile(zip_path: str) -> gpd.GeoDataFrame:
    """Read a shapefile stored inside a .zip using GeoPandas."""
    if not os.path.exists(zip_path):
        raise FileNotFoundError(f"File not found: {zip_path}")
    return gpd.read_file(f"zip://{zip_path}")


def read_file_geodatabase(gdb_path: str, layer: str | None = None) -> gpd.GeoDataFrame:
    """Read a layer from an Esri File Geodatabase (.gdb). If layer is None,
    use the first layer listed in the database."""
    if not os.path.exists(gdb_path):
        raise FileNotFoundError(f"File not found: {gdb_path}")
    
    import pyogrio  # local import so this function works even if pyogrio is not used elsewhere
    
    layers = pyogrio.list_layers(gdb_path)
    if layer is None:
        # pick the first layer by default
        layer = layers[0][0]
    
    print(f"Reading GDB layer '{layer}' from {gdb_path}")
    return gpd.read_file(gdb_path, layer=layer)


# Load building footprints from zipped shapefiles
building_current_gdf = read_zipped_shapefile(PATH_BUILDING_CURRENT_ZIP)
building_historic_gdf = read_zipped_shapefile(PATH_BUILDING_HISTORIC_ZIP)

# Load stormwater from FileGDB
STORMWATER_LAYER_NAME = "NYC_Stormwater_Flood_Map_Moderate_Flood_2_13_inches_per_hr_with_Current_Sea_Levels"

stormwater_gdf = read_file_geodatabase(
    PATH_STORMWATER_GDB,
    layer=STORMWATER_LAYER_NAME
)

print("Loaded GeoDataFrames:")
print("  building_current:", building_current_gdf.shape)
print("  building_historic:", building_historic_gdf.shape)
print("  stormwater:", stormwater_gdf.shape)


Reading GDB layer 'NYC_Stormwater_Flood_Map_Moderate_Flood_2_13_inches_per_hr_with_Current_Sea_Levels' from c:\Code\NYCDSA\nyc-urban-growth-planimetrics\data\raw\stormwater\stormwater_moderate_current.gdb


c:\Code\NYCDSA\nyc-urban-growth-planimetrics\.venv\Lib\site-packages\pyogrio\raw.py:200: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_read(


Loaded GeoDataFrames:
  building_current: (1082999, 17)
  building_historic: (49059, 18)
  stormwater: (2, 4)


In [5]:
datasets = [
    ("building_current", building_current_gdf),
    ("building_historic", building_historic_gdf),
    ("stormwater", stormwater_gdf),
]

for name, gdf in datasets:
    inspect_geodf(gdf, name)


DATASET: building_current
Type: <class 'geopandas.geodataframe.GeoDataFrame'>
Number of rows: 1,082,999
Number of columns: 17

CRS: EPSG:4326

Geometry types:
Polygon    1082999
Name: count, dtype: int64

Columns and dtypes:
name                  object
bin                  float64
doitt_id             float64
shape_area           float64
base_bbl              object
objectid             float64
constructi           float64
feature_co           float64
geom_sourc            object
ground_ele           float64
height_roo           float64
date_last_    datetime64[ms]
time_last_            object
last_statu            object
mappluto_b            object
shape_leng           float64
geometry            geometry
dtype: object

First few rows:


,name,bin,doitt_id,shape_area,base_bbl,objectid,constructi,feature_co,geom_sourc,ground_ele,height_roo,date_last_,time_last_,last_statu,mappluto_b,shape_leng,geometry
0,None,4451699.0,321944.0,177.746094,4075320028,507357.0,1950.0,2100.0,Other (Manual),93.0,27.000000,2017-08-22,19:18:38.000,Constructed,4075327501,59.004939,"POLYGON ((-73.75416 40.7542, -73.75402 40.7542..."
1,None,4558952.0,255026.0,34.742188,4105630045,137879.0,1930.0,5110.0,Photogrammetric,72.0,13.060000,2017-08-17,16:20:43.000,Constructed,4105630045,24.548387,"POLYGON ((-73.75283 40.71895, -73.75289 40.718..."
2,None,3176483.0,759005.0,180.890625,3066450044,982953.0,1915.0,2100.0,Photogrammetric,18.0,36.761589,2017-08-22,15:37:34.000,Constructed,3066450044,61.475641,"POLYGON ((-73.98372 40.60334, -73.98381 40.603..."



Null counts (top 15 columns by null count):


name          1080751
constructi      10120
ground_ele        554
last_statu        383
geom_sourc        347
base_bbl            0
shape_area          0
bin                 0
doitt_id            0
feature_co          0
objectid            0
date_last_          0
height_roo          0
time_last_          0
mappluto_b          0
dtype: int64


No obvious categorical fields found among standard names; you may want to add custom checks.
DATASET: building_historic
Type: <class 'geopandas.geodataframe.GeoDataFrame'>
Number of rows: 49,059
Number of columns: 18

CRS: EPSG:4326

Geometry types:
Polygon    49059
Name: count, dtype: int64

Columns and dtypes:
name                  object
bin                  float64
doitt_id             float64
shape_area           float64
base_bbl              object
objectid             float64
constructi           float64
demolition           float64
feature_co           float64
geom_sourc            object
ground_ele           float64
height_roo           float64
date_last_    datetime64[ms]
time_last_            object
last_statu            object
mappluto_b            object
shape_leng           float64
geometry            geometry
dtype: object

First few rows:


...


No obvious categorical fields found among standard names; you may want to add custom checks.
DATASET: stormwater
Type: <class 'geopandas.geodataframe.GeoDataFrame'>
Number of rows: 2
Number of columns: 4

CRS: EPSG:2263

Geometry types:
MultiPolygon    2
Name: count, dtype: int64

Columns and dtypes:
Flooding_Category       int16
Shape_Length          float64
Shape_Area            float64
geometry             geometry
dtype: object

First few rows:


,Flooding_Category,Shape_Length,Shape_Area,geometry
0,1,6.379481e+06,7.807795e+07,"MULTIPOLYGON Z (((917257.748 120888.12 0, 9172..."
1,2,2.585630e+06,4.012622e+07,"MULTIPOLYGON Z (((917241.716 120860.909 0, 917..."



Null counts (top 15 columns by null count):


Flooding_Category    0
Shape_Length         0
Shape_Area           0
geometry             0
dtype: int64


No obvious categorical fields found among standard names; you may want to add custom checks.


In [6]:
print("CRS – building_current:", building_current_gdf.crs)
print("CRS – building_historic:", building_historic_gdf.crs)
print("CRS – stormwater:", stormwater_gdf.crs)


CRS – building_current: EPSG:4326
CRS – building_historic: EPSG:4326
CRS – stormwater: EPSG:2263


In [7]:
current_cols = set(building_current_gdf.columns)
historic_cols = set(building_historic_gdf.columns)

print("Columns only in CURRENT:")
for col in sorted(current_cols - historic_cols):
    print("  ", col)

print("\nColumns only in HISTORIC:")
for col in sorted(historic_cols - current_cols):
    print("  ", col)

print("\nColumns in common:", len(current_cols & historic_cols))


Columns only in CURRENT:

Columns only in HISTORIC:
   demolition

Columns in common: 17


In [8]:
# Target projected CRS for spatial analysis and area calculations
TARGET_CRS = "EPSG:2263"  # NAD83 / New York Long Island (ft US)

print("Target CRS:", TARGET_CRS)


Target CRS: EPSG:2263


In [9]:
def reproject_to_target(gdf: gpd.GeoDataFrame, target_crs: str) -> gpd.GeoDataFrame:
    """Reproject a GeoDataFrame to target_crs if needed."""
    if gdf.crs is None:
        raise ValueError("GeoDataFrame has no CRS set.")
    if gdf.crs.to_string() == target_crs:
        print("Already in target CRS:", target_crs)
        return gdf
    print(f"Reprojecting from {gdf.crs} to {target_crs}...")
    return gdf.to_crs(target_crs)


# Reproject building datasets to match stormwater CRS (2263, feet)
building_current_2263 = reproject_to_target(building_current_gdf, TARGET_CRS)
building_historic_2263 = reproject_to_target(building_historic_gdf, TARGET_CRS)

# Stormwater is already in 2263, so we just alias it for consistency
stormwater_2263 = stormwater_gdf  # no reprojection needed

print("CRS after reprojection:")
print("  building_current_2263:", building_current_2263.crs)
print("  building_historic_2263:", building_historic_2263.crs)
print("  stormwater_2263:", stormwater_2263.crs)


Reprojecting from EPSG:4326 to EPSG:2263...
Reprojecting from EPSG:4326 to EPSG:2263...
CRS after reprojection:
  building_current_2263: EPSG:2263
  building_historic_2263: EPSG:2263
  stormwater_2263: EPSG:2263


In [10]:
def add_geom_area(gdf: gpd.GeoDataFrame, area_col_name: str = "geom_area_sqft") -> gpd.GeoDataFrame:
    """Compute area from geometry (in square feet, since CRS is 2263)
    and store in a new column."""
    if gdf.crs.to_string() != "EPSG:2263":
        raise ValueError("add_geom_area expects CRS=EPSG:2263 so that units are feet.")
    gdf = gdf.copy()
    gdf[area_col_name] = gdf.geometry.area
    return gdf


# Add geometry-based area to the reprojected building datasets
building_current_2263 = add_geom_area(building_current_2263)
building_historic_2263 = add_geom_area(building_historic_2263)

building_current_2263[["shape_area", "geom_area_sqft"]].head()


,shape_area,geom_area_sqft
0,177.746094,1096.759439
1,34.742188,214.631781
2,180.890625,1121.202926
3,106.035156,656.015069
4,217.175781,1334.183950


In [11]:
def compare_area_fields(gdf: gpd.GeoDataFrame, label: str):
    print("=" * 60)
    print("AREA COMPARISON:", label)
    print("=" * 60)
    print(gdf[["shape_area", "geom_area_sqft"]].describe())
    
    # relative difference (avoid divide-by-zero)
    eps = 1e-9
    rel_diff = (gdf["geom_area_sqft"] - gdf["shape_area"]) / (gdf["shape_area"] + eps)
    print("\nRelative difference (geom_area_sqft vs shape_area):")
    print(rel_diff.describe())


compare_area_fields(building_current_2263, "building_current_2263")
compare_area_fields(building_historic_2263, "building_historic_2263")


AREA COMPARISON: building_current_2263
         shape_area  geom_area_sqft
count  1.082999e+06    1.082999e+06
mean   2.604568e+02    1.609370e+03
std    9.169336e+02    5.663732e+03
min    5.246094e+00    3.247651e+01
25%    1.057734e+02    6.539524e+02
50%    1.552070e+02    9.593253e+02
75%    2.141211e+02    1.323959e+03
max    1.903241e+05    1.171875e+06

Relative difference (geom_area_sqft vs shape_area):
count    1.082999e+06
mean     5.181844e+00
std      1.514027e-02
min      5.140030e+00
25%      5.172186e+00
50%      5.182742e+00
75%      5.193569e+00
max      5.218737e+00
dtype: float64
AREA COMPARISON: building_historic_2263
          shape_area  geom_area_sqft
count   49059.000000    49059.000000
mean      398.267228     2460.895127
std      1682.286158    10398.866956
min         5.605469       34.571196
25%        98.437500      608.753523
50%       166.421875     1028.798896
75%       271.314453     1676.659974
max    101077.062500   625767.853765

Relative difference

In [12]:
def print_bounds(gdf: gpd.GeoDataFrame, name: str):
    minx, miny, maxx, maxy = gdf.total_bounds
    print(f"{name} bounds:")
    print(f"  minx={minx:.2f}, miny={miny:.2f}, maxx={maxx:.2f}, maxy={maxy:.2f}")
    print()

print_bounds(building_current_2263, "building_current_2263")
print_bounds(building_historic_2263, "building_historic_2263")
print_bounds(stormwater_2263, "stormwater_2263")


building_current_2263 bounds:
  minx=913201.22, miny=120963.25, maxx=1067366.79, maxy=272672.09

building_historic_2263 bounds:
  minx=913437.49, miny=120953.84, maxx=1067462.88, maxy=271841.46

stormwater_2263 bounds:
  minx=913848.42, miny=120828.05, maxx=1065777.82, maxy=271240.93



In [ ]:
# Small random sample of buildings to test the join logic
sample_size = 10000
building_sample = building_current_2263.sample(
    min(sample_size, len(building_current_2263)), random_state=42
)

# Spatial join: which sampled buildings fall inside stormwater polygons?
joined_sample = gpd.sjoin(
    building_sample,
    stormwater_2263[["Flooding_Category", "geometry"]],
    how="left",
    predicate="intersects",
)

print("Sample size:", len(building_sample))
print("Joined rows:", len(joined_sample))

print("\nFlooding category value counts on sample:")
print(joined_sample["Flooding_Category"].value_counts(dropna=False))


In [ ]:
import pyarrow
print(pyarrow.__version__)


22.0.0


In [ ]:
DATA_PROCESSED = os.path.join("..", "data", "processed")
os.makedirs(DATA_PROCESSED, exist_ok=True)

building_current_2263.to_parquet(os.path.join(DATA_PROCESSED, "building_current_2263.parquet"))
building_historic_2263.to_parquet(os.path.join(DATA_PROCESSED, "building_historic_2263.parquet"))
stormwater_2263.to_parquet(os.path.join(DATA_PROCESSED, "stormwater_2263.parquet"))
